In [125]:
import json
import re

import requests
import scrapy

#importing everything that I need

In [126]:
#getting the url for the website
base_url = 'http://goheels.com'
#getting the url for the actual roster
url = base_url + '/roster.aspx?path=baseball'
#making sure they let us actually get the data by showing UNC that we are students
headers = {'User-Agent': 'UNC Journo Class'}
#GETTING the url and the headers and requesting the access for the info
resp = requests.get(url, headers={'User-Agent': 'UNC Journo Class'},)

In [127]:
#once we get to the content, we want to store the data we get (right now it is in bytes instead of strings)
body_bytes = resp.content
#we want to make sure that our data is in string form instead of byte form, so we convert it here
body_str = body_bytes.decode('utf-8')
#let's take a look at the data we have found
body_str

'\r\n\r\n<!doctype html>\r\n<html id="ctl00_html" lang="en" class=" wide baseball sport baseball"> \r\n    <head><script>window.client_hostname = "goheels.com";window.server_name = "866411-LB16";window.client_division = "DI";window.targetting_criteria = {"page_template":"roster-list","sport_name":"baseball","sport_name_custom":"baseball","site":"unc","content_id":null,"sect":"mbasebl"};window.client_shortname = "unc";window.dfp_network_code = "29658103";window.img_dfp_unit_name = "";window.block_dfp = false;window.reactions = null;</script><link rel="stylesheet" href="/less/sidearm-responsive-grid.less?_=636518877046678376"><link rel="stylesheet" href="https://s3.amazonaws.com/assets.sidearmsports.com/adaptive/css/normalize.min.css"><link rel="apple-touch-icon" href="/images/logos/site/site.png"><link rel="canonical" href="http://goheels.com/roster.aspx?roster=498&amp;path=baseball"><meta charset="utf-8"><meta name="viewport" content="width=device-width"><link rel="apple-touch-icon" hr

In [128]:
#we now want to select the info that we have collected in order to do stuff with it!
sel = scrapy.Selector(text=body_str)

In [129]:
#now we want to find the table that we are going to work on
table = sel.css('table')[0]

In [130]:
#let's see what the table is
table

<Selector xpath='descendant-or-self::table' data='<table class="sidearm-table sidearm-tabl'>

In [131]:
table.css('tr') # Get all the tr elements within the table
table.css('tr')[0] # Get the first tr element within the table using the pythonic index
table.css('tr')[0].css('th') # Get all the th elements from within the first tr
table.css('tr:nth-child(1) th')

[<Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' data='<th scope="col" class="text-left text-up'>,
 <Selector xpath='descendant-or-self::tr[count(preceding-sibling::*) = 0]/descendant-or-self::*/th' 

In [132]:
#now we want to get the column names in order to put the information into them according to their category
columns = table.css('th').xpath('string()').extract()

In [133]:
#let's see what the columns actually are
columns

['#',
 'Full Name',
 'Pos.',
 'Ht.',
 'Wt.',
 'Academic Year',
 'Hometown / High School']

In [134]:
#okay, so we've got the columns. Now let's get the rows. Keep in mind that we only want the 1st and on because the 0 row is all the names of the columns (we already have that)
rows = table.css('tr')[1:]

In [135]:
#okay, we've got the data. Now we need to organize it to make sure that everything comes up in a list. we made a variable by_player_number to become a list [].
by_player_number = []
#now, we need to make sure that for every row in rows, something happens. So we will create a loop to go through all of the players
for r in rows:
    #we put all that data in this DICTIONARY {} called data
    data = {}
    #i is just any variable and d is every cell (td basically). we want to see how much of everything we have, so we enumerate it.
    #once you understand everything, add more info here
    for i, d in enumerate(r.css('td')):
        #a is another variable that we are going to use for links. so go into the data from the cell, and get the link
        a = d.css('a')
        #if there is a link.....
        if a:
            #a new varible equals the text that comes from the xpath of a. 
            t = a.xpath('text()').extract()[0]
            #go to the href in data (defined above) and grab the url from the link
            data['href'] = a.xpath('@href').extract()[0]
        #if there's not a link....
        else:
            #that variable will be the text from the cell
            t = d.xpath('text()').extract()[0]
        #go to the data, go to each data in the column and that will equal t ???????????????????????????
        data[columns[i]] = t
    #go to by_player_number and add the data to the dictionary    
    by_player_number.append(data)

In [136]:
by_player_number 
#let's look at what our dictionary has collected and displayed, within the list by_player_number

[{'#': '1',
  'Academic Year': 'Jr.',
  'Full Name': 'Brandon Riley',
  'Hometown / High School': 'Burlington, N.C. / Williams',
  'Ht.': '6-0',
  'Pos.': 'OF',
  'Wt.': '175',
  'href': '/roster.aspx?rp_id=14221'},
 {'#': '2',
  'Academic Year': 'Fr.',
  'Full Name': 'Satchel Jerzembeck',
  'Hometown / High School': 'Charlotte, N.C. / Providence',
  'Ht.': '5-10',
  'Pos.': 'IF',
  'Wt.': '150',
  'href': '/roster.aspx?rp_id=14226'},
 {'#': '3',
  'Academic Year': 'Jr.',
  'Full Name': 'Kyle Datres',
  'Hometown / High School': 'Williamsport, Pa. / Loyalsock Township',
  'Ht.': '6-0',
  'Pos.': '3B',
  'Wt.': '198',
  'href': '/roster.aspx?rp_id=14211'},
 {'#': '4',
  'Academic Year': 'So.',
  'Full Name': 'Brandon Martorano',
  'Hometown / High School': 'Marlboro, N.J. / Christian Brothers Academy',
  'Ht.': '6-2',
  'Pos.': 'C/OF',
  'Wt.': '187',
  'href': '/roster.aspx?rp_id=14219'},
 {'#': '5',
  'Academic Year': 'So.',
  'Full Name': 'Ashton McGee',
  'Hometown / High School': '

In [137]:
#we've got the basic info, but now we need to go inside the individual player's page and get their bio information. Keep in mind that player is temporary, solely within this loop.
def fetch_bio(player):
    #let's find the player's individual url so we can go inside of it
    player_url = base_url + player['href']
    print('Fetch bio', player_url)
    #okay, let's request to go inside of this page to get into the other page
    resp = requests.get(player_url, headers=headers)
    #let's decode the byte into ???strings??? instead of bytes
    player_txt = resp.content.decode('utf-8')
    #let's select that data within this page
    sel = scrapy.Selector(text=player_txt)
    player['sel'] = sel
    #go to each player, find the bio, which is defined and found here
    player['bio'] = sel.css('#sidearm-roster-player-bio').xpath('string()').extract()[0]
    #go to each player, find the image, which is defined and found here
    player['img'] = sel.css('.sidearm-roster-player-image img').xpath('@src').extract()[0]

In [138]:
#I know this is regex, but i'm not sure what the compiling is doing...
js_obj_rx = re.compile(r'.*?responsive-roster-bio\.ashx.*?(?P<obj>{.*?})')

In [139]:
#okay, now we are going to fetch the stats for each player. So we are going to make a loop :
def fetch_stats(player):
    #to get the text, go to the player, get all the info from that stats page, get the string of info and extract it
    text = player['sel'].xpath('string()').extract()[0]
    #now we are going to split it for some reason???????????????????
    parts = text.split('$.getJSON("/services/')[1:]
    #regex, still don't understand fully but we are getting all the parts closer together i believe
    captured = js_obj_rx.findall(''.join(parts))
    #now, we are making a list to put the info in
    clean_objs = []
    #??????????????????????????????????????? another loop??????????
    for obj_str in captured:
        # We only want the stats object...
        if 'stats' not in obj_str:
            continue

        obj_str = obj_str.replace('{', '').replace('}', '')
        obj_str = obj_str.replace("'", '').replace('"', '')
        obj_pairs = obj_str.split(',')
        obj_pairs = [x.split(":") for x in obj_pairs]
        clean_pairs = []
        for pair in obj_pairs:
            clean_pairs.append(['"{}"'.format(p.strip()) for p in pair])
        colonized = [":".join(p) for p in clean_pairs]
        commas = ','.join(colonized)
        json_str = "{" + commas + "}"
        clean_objs.append(json.loads(json_str))
    
    player['stats_url'] = stats_url = (
        "http://goheels.com/services/responsive-roster-bio.ashx?"
        "type={type}&rp_id={rp_id}&path={path}&year={year}"
        "&player_id={player_id}"
    ).format(**clean_objs[0])
    
    print('Fetch stats', stats_url)
    resp = requests.get(stats_url, headers=headers)
    json_stats = json.loads(resp.content.decode("utf-8"))
    player['raw_stats'] = json_stats

In [140]:
for p in by_player_number:
    fetch_bio(p)
    fetch_stats(p)
    
 #p is temporary, basically means for every player

Fetch bio http://goheels.com/roster.aspx?rp_id=14221
Fetch stats http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id=14221&path=baseball&year=2018&player_id=0
Fetch bio http://goheels.com/roster.aspx?rp_id=14226
Fetch stats http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id=14226&path=baseball&year=2018&player_id=0
Fetch bio http://goheels.com/roster.aspx?rp_id=14211
Fetch stats http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id=14211&path=baseball&year=2018&player_id=0
Fetch bio http://goheels.com/roster.aspx?rp_id=14219
Fetch stats http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id=14219&path=baseball&year=2018&player_id=0
Fetch bio http://goheels.com/roster.aspx?rp_id=14220
Fetch stats http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id=14220&path=baseball&year=2018&player_id=0
Fetch bio http://goheels.com/roster.aspx?rp_id=14228
Fetch stats http://goheels.com/services/responsive-r

In [141]:
by_player_number[0]
#how did the bio and stats get appended to the rest of the keys

{'#': '1',
 'Academic Year': 'Jr.',
 'Full Name': 'Brandon Riley',
 'Hometown / High School': 'Burlington, N.C. / Williams',
 'Ht.': '6-0',
 'Pos.': 'OF',
 'Wt.': '175',
 'bio': '\r\n                        Biography\r\n                                                    \r\n                            Dynamic outfielder who figures to be among the team leaders as a junior.\r\n\r\nSophomore Season (2017)\r\nHit .317 with seven homers, nine doubles and a team-high 52 RBIs •\xa0Also scored 46 runs, drew 37 walks and led the team with five triples, which is tied for 10th-most in UNC history for a single season • Appeared in all 63 games and started 62, primarily in right field • Stole 10 bases in 14 attempts • Had 21 multi-hit games, including a season-high three hits on eight occasions • Also drove in multiple runs 14 times, including a season-best four at ECU on March 22 • Followed up the ECU game with a big series against Miami, recording six hits and driving in four runs with a homer 

In [142]:
p = [p for p in by_player_number if p['Full Name'] == 'Brandon Riley'][0]
#this is just to show the first guy

In [143]:
txt = p['raw_stats']['career_stats']
#text equals for every player, go into raw stats and then go into career stats and get that info

#let's see what we have selected
print(txt)


            <section>
                <h5>Hitting Statistics</h5>

                <div class="sidearm-table-overflow-on-x-large">
                    <table class="sidearm-table highlight-column-hover">
                        <caption>Career Hitting Statistics</caption>
                        <thead>
                            <tr>
                                <th scope="col" class="text-hide">Season</th>
                                <th scope="col">AVG</th>
                                <th scope="col">GP</th>
                                <th scope="col">GS</th>
                                <th scope="col">AB</th>
                                <th scope="col">R</th>
                                <th scope="col">H</th>
                                <th scope="col">2B</th>
                                <th scope="col">3B</th>
                                <th scope="col">HR</th>
                                <th scope="col">RBI</th>
                       

In [144]:
#again, let's select the variable txt and make it text
sel = scrapy.Selector(text=txt)

In [145]:
#go into the selected content and go to the section within the code that holds all the stats we want
sel.css('section')

[<Selector xpath='descendant-or-self::section' data='<section>\r\n                <h5>Hitting S'>]

In [ ]:
#as of now, I believe parsing is making our info clean. This is what we are going to do now so it makes sense to humans
def parse_stats(player):
    stats = {}
    for raw_key, raw_val in player['raw_stats'].items():
        txt = player['raw_stats'][raw_key]
        if not txt:
            print('Skipping {} for {}'.format(raw_key, player['Full Name']))
            continue
        sel = scrapy.Selector(text=txt)
        # Get all the tables
        for section in sel.css('section'):
            title = section.css('h5').xpath('string()').extract()[0]
            cols = section.css('tr')[0].css('th').xpath('string()').extract()
            print('TITLE OF STATS TABLE', title)
            print('COLUMNS', cols)
            these_stats = []
            print('ROWS', section.css('tr'))
            for r in section.css('tr')[1:]:
                print('row', r.xpath('string()').extract()[0].replace('\r', '').replace('\n', '').strip())
                s = {}
                for i, d in enumerate(r.css('td'), 1):
                    s[cols[i].lower()] = d.xpath('string()').extract()[0]
                yr = r.css('th').xpath('string()')
                if yr:
                    yr = yr.extract()[0]
                    if yr.lower() in ('total', 'season'):
                        print('SKIPPING...')
                        continue
                    print('THE YEAR IS', yr)
                    s['yr'] = yr
                these_stats.append(s)
                print('THE STATS ARE', these_stats)
            existing = stats.get(raw_key, {})
            existing[title] = these_stats
            stats[raw_key] = existing
    player['stats'] = stats

In [147]:
p = [p for p in by_player_number if p['Full Name'] == 'Brandon Riley'][0]
parse_stats(p)
#now we are parsing the stats and just showing the first guy...just so we know if it works

Skipping current_stats for Brandon Riley
TITLE OF STATS TABLE Hitting Statistics
COLUMNS ['Season', 'AVG', 'GP', 'GS', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SLG%', 'BB', 'HBP', 'SO', 'GDP', 'OB%', 'SF', 'SH', 'SB', 'PO', 'A', 'E', 'FLD%']
ROWS [<Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                <t'>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                <t'>]
row 2016                                    .294                                    54                                    51                                    194                                    27                                    57                                    16                                    3                                    1                            

In [148]:
for p in by_player_number:
    parse_stats(p)
    
#now let's look at everyone's stats!

Skipping current_stats for Brandon Riley
TITLE OF STATS TABLE Hitting Statistics
COLUMNS ['Season', 'AVG', 'GP', 'GS', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SLG%', 'BB', 'HBP', 'SO', 'GDP', 'OB%', 'SF', 'SH', 'SB', 'PO', 'A', 'E', 'FLD%']
ROWS [<Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                <t'>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                <t'>]
row 2016                                    .294                                    54                                    51                                    194                                    27                                    57                                    16                                    3                                    1                            

row Total                                .250                                165                                144                                511                                88                                128                                19                                1                                14                                108                                .374                                69                                15                                112                                10                                .352                                8                                11                                10                                575                                174                                27                                .965
SKIPPING...
Skipping gamehigh_stats for Zack Gahagan
Skipping current_stats for Cody Roberts
TITLE OF STATS TABLE Pitching Statistics
COLUMNS ['Season', 'ERA', 'W', 'L', 'APP', 'GS', 'CG', 'SHO'

THE YEAR IS 2017
THE STATS ARE [{'avg': 'NaN', 'gp': '16', 'gs': '15', 'ab': '0', 'r': '0', 'h': '0', '2b': '0', '3b': '0', 'hr': '0', 'rbi': '0', 'slg%': '.000', 'bb': '0', 'hbp': '0', 'so': '0', 'gdp': '0', 'ob%': '.000', 'sf': '0', 'sh': '0', 'sb': '0', 'po': '5', 'a': '6', 'e': '1', 'fld%': '.917', 'yr': '2017'}]
row Total                                .000                                16                                15                                0                                0                                0                                0                                0                                0                                0                                .000                                0                                0                                0                                0                                .000                                0                                0                                0                                5 

SKIPPING...
Skipping gamehigh_stats for Hansen Butler
Skipping current_stats for Taylor Sugg
TITLE OF STATS TABLE Pitching Statistics
COLUMNS ['Season', 'ERA', 'W', 'L', 'APP', 'GS', 'CG', 'SHO', 'SV', 'IP', 'H', 'R', 'ER', 'BB', 'SO', '2B', '3B', 'HR', 'BF', 'BAVG', 'WP', 'HBP', 'BK', 'SFA', 'SHA']
ROWS [<Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                <t'>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                <t'>]
row 2016                                    2.45                                    0                                    0                                    14                                    0                                    0                                    0                                    1             

row Total                                3.65                                1                                0                                16                                0                                0                                0                                0                                12.1                                11                                7                                5                                7                                14                                4                                0                                0                                55                                .244                                2                                2                                0                                0                                1
SKIPPING...
TITLE OF STATS TABLE Hitting Statistics
COLUMNS ['Season', 'AVG', 'GP', 'GS', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SLG%', 'BB', 'HBP', 'SO', 'GDP', 'OB%', 'SF', 'SH', '

ROWS [<Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                <t'>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>, <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                <t'>]
row 2017                                    NaN                                    27                                    3                                    0                                    0                                    0                                    0                                    0                                    0                                    0                                    .000                                    0                                    0                                    0                                    0                                    .000                                    0                                    0                                

In [149]:
print(p)
temp_player = [p for p in by_player_number if p['Full Name'] == 'Brandon Riley'][0]

print(temp_player)
#AND WHY IS 'STATS' A KEY ERROR - ITS DEFINED ABOVE

{'#': '48', 'href': '/roster.aspx?rp_id=14216', 'Full Name': 'Rodney Hutchison Jr.', 'Pos.': 'RHP', 'Ht.': '6-6', 'Wt.': '239', 'Academic Year': 'Jr.', 'Hometown / High School': 'Mason, Ohio / Mason', 'sel': <Selector xpath=None data='<html id="ctl00_html" lang="en" class=" '>, 'bio': '\r\n                        Biography\r\n                                                    \r\n                            Sophomore Season (2017)\r\nDurable righty who emerged as a key bullpen piece as a sophomore • Made 31 total appearances, including 30 in relief, and finished 7-4 with a 4.14 ERA and 53 strikeouts • Issued just 17 walks in 58.2 innings, including a team-high 57.0 relief innings • Went at least two innings in relief on 13 occasions, including a season-high 5.0 innings against Miami on March 26 • Worked three shutout frames to get the win over Miami in the ACC tournament • Had 15 multi-strikeout appearances, including a season-high five punchouts four times • Struck out four and allow

KeyError: 'Name'

In [150]:
by_player_number

[{'#': '1',
  'Academic Year': 'Jr.',
  'Full Name': 'Brandon Riley',
  'Hometown / High School': 'Burlington, N.C. / Williams',
  'Ht.': '6-0',
  'Pos.': 'OF',
  'Wt.': '175',
  'bio': '\r\n                        Biography\r\n                                                    \r\n                            Dynamic outfielder who figures to be among the team leaders as a junior.\r\n\r\nSophomore Season (2017)\r\nHit .317 with seven homers, nine doubles and a team-high 52 RBIs •\xa0Also scored 46 runs, drew 37 walks and led the team with five triples, which is tied for 10th-most in UNC history for a single season • Appeared in all 63 games and started 62, primarily in right field • Stole 10 bases in 14 attempts • Had 21 multi-hit games, including a season-high three hits on eight occasions • Also drove in multiple runs 14 times, including a season-best four at ECU on March 22 • Followed up the ECU game with a big series against Miami, recording six hits and driving in four runs with 

In [152]:
[p for p in by_player_number if p['Full Name'] == 'Brandon Riley'][0]['stats']

{'career_stats': {'Hitting Statistics': [{'2b': '16',
    '3b': '3',
    'a': '0',
    'ab': '194',
    'avg': '.294',
    'bb': '20',
    'e': '1',
    'fld%': '.929',
    'gdp': '3',
    'gp': '54',
    'gs': '51',
    'h': '57',
    'hbp': '1',
    'hr': '1',
    'ob%': '.355',
    'po': '13',
    'r': '27',
    'rbi': '35',
    'sb': '3',
    'sf': '5',
    'sh': '2',
    'slg%': '.423',
    'so': '28',
    'yr': '2016'},
   {'2b': '9',
    '3b': '5',
    'a': '1',
    'ab': '224',
    'avg': '.317',
    'bb': '37',
    'e': '4',
    'fld%': '.964',
    'gdp': '6',
    'gp': '63',
    'gs': '62',
    'h': '71',
    'hbp': '0',
    'hr': '7',
    'ob%': '.406',
    'po': '106',
    'r': '46',
    'rbi': '52',
    'sb': '10',
    'sf': '5',
    'sh': '2',
    'slg%': '.496',
    'so': '34',
    'yr': '2017'}]}}

In [153]:
to_dump = [p.copy() for p in by_player_number]
for p in to_dump:
    p.pop('sel')
    for k in list(p.keys()):
        if 'raw' in k:
            p.pop(k)
with open('baseball_players_scraped.json', 'w') as f:
    json.dump(to_dump, f)
    
#ok fam! Now we are going to make this into a json file! So, we need to first create a json file to dump the info into, and then we are going to literally dump the info into that file

In [154]:
cat scraped_players.json | cut -c 1-100

[{"#": "1", "href": "/roster.aspx?rp_id=14221", "Full Name": "Brandon Riley", "Pos.": "OF", "Ht.": "


In [155]:
to_dump[0]
#let's see what we are dumping for one player

{'#': '1',
 'Academic Year': 'Jr.',
 'Full Name': 'Brandon Riley',
 'Hometown / High School': 'Burlington, N.C. / Williams',
 'Ht.': '6-0',
 'Pos.': 'OF',
 'Wt.': '175',
 'bio': '\r\n                        Biography\r\n                                                    \r\n                            Dynamic outfielder who figures to be among the team leaders as a junior.\r\n\r\nSophomore Season (2017)\r\nHit .317 with seven homers, nine doubles and a team-high 52 RBIs •\xa0Also scored 46 runs, drew 37 walks and led the team with five triples, which is tied for 10th-most in UNC history for a single season • Appeared in all 63 games and started 62, primarily in right field • Stole 10 bases in 14 attempts • Had 21 multi-hit games, including a season-high three hits on eight occasions • Also drove in multiple runs 14 times, including a season-best four at ECU on March 22 • Followed up the ECU game with a big series against Miami, recording six hits and driving in four runs with a homer 

In [167]:
import numpy as np
import pandas as pd
#import these things we need for data frames and working within them

In [168]:
#json, open my json file I made with my info, and then load it so we can work with it
data = json.load(open('baseball_players_scraped.json'))

In [169]:
#look at the info in our json file for the first player
data[0]

{'#': '1',
 'Academic Year': 'Jr.',
 'Full Name': 'Brandon Riley',
 'Hometown / High School': 'Burlington, N.C. / Williams',
 'Ht.': '6-0',
 'Pos.': 'OF',
 'Wt.': '175',
 'bio': '\r\n                        Biography\r\n                                                    \r\n                            Dynamic outfielder who figures to be among the team leaders as a junior.\r\n\r\nSophomore Season (2017)\r\nHit .317 with seven homers, nine doubles and a team-high 52 RBIs •\xa0Also scored 46 runs, drew 37 walks and led the team with five triples, which is tied for 10th-most in UNC history for a single season • Appeared in all 63 games and started 62, primarily in right field • Stole 10 bases in 14 attempts • Had 21 multi-hit games, including a season-high three hits on eight occasions • Also drove in multiple runs 14 times, including a season-best four at ECU on March 22 • Followed up the ECU game with a big series against Miami, recording six hits and driving in four runs with a homer 

In [174]:
#we are going to make our data that will go into the data frame eventually a list, because right now it isn't in a format that we can use to put into a data frame
data_for_future_df = []

In [175]:
for player in data:
    for key, val in player.get('stats', {}).get('career_stats', {}).items():
        stat_type = key.lower().split()[0]
        name = player.get('Full Name')
        if not name:
            continue
        num = player.get('#')
        if not num:
            continue
            
        for stat in val:
            # Add the name
            stat['name'] = name
            # Add the player number
            stat['num'] = num
            # Add the stat_type
            stat['stat_type'] = stat_type
            # Append the denormalized row
            # This is now something like
            # {'name': 'John Doe', 'num': '1', 'stat_type': 'hitting', 'year': '2016', ...}
            data_for_future_df.append(stat)

In [176]:
#let's define the dataframe variable so we can look at it
df = pd.DataFrame(data_for_future_df)

In [178]:
#let's look at the top five rows of our data frame, just to get an idea of how it looks
df.head()

,2b,3b,a,ab,app,avg,bavg,bb,bf,bk,...,sh,sha,sho,slg%,so,stat_type,sv,w,wp,yr
0,16,3,0,194,NaN,.294,NaN,20,NaN,NaN,...,2,NaN,NaN,.423,28,hitting,NaN,NaN,NaN,2016
1,9,5,1,224,NaN,.317,NaN,37,NaN,NaN,...,2,NaN,NaN,.496,34,hitting,NaN,NaN,NaN,2017
2,6,2,64,152,NaN,.250,NaN,27,NaN,NaN,...,7,NaN,NaN,.375,28,hitting,NaN,NaN,NaN,2016
3,13,1,109,230,NaN,.265,NaN,29,NaN,NaN,...,4,NaN,NaN,.422,45,hitting,NaN,NaN,NaN,2017
4,0,0,12,44,NaN,.068,NaN,6,NaN,NaN,...,2,NaN,NaN,.068,13,hitting,NaN,NaN,NaN,2017


In [179]:
#now let's apply it and if there are errors, ignore it
df = df.apply(pd.to_numeric, errors='ignore')

In [180]:
#okay, we don't want the average so let's drop it
df.drop('avg', axis=1, inplace=True)

In [181]:
#see the first five results
df.head()

,2b,3b,a,ab,app,bavg,bb,bf,bk,cg,...,sh,sha,sho,slg%,so,stat_type,sv,w,wp,yr
0,16,3,0.0,194.0,NaN,NaN,20,NaN,NaN,NaN,...,2.0,NaN,NaN,0.423,28,hitting,NaN,NaN,NaN,2016
1,9,5,1.0,224.0,NaN,NaN,37,NaN,NaN,NaN,...,2.0,NaN,NaN,0.496,34,hitting,NaN,NaN,NaN,2017
2,6,2,64.0,152.0,NaN,NaN,27,NaN,NaN,NaN,...,7.0,NaN,NaN,0.375,28,hitting,NaN,NaN,NaN,2016
3,13,1,109.0,230.0,NaN,NaN,29,NaN,NaN,NaN,...,4.0,NaN,NaN,0.422,45,hitting,NaN,NaN,NaN,2017
4,0,0,12.0,44.0,NaN,NaN,6,NaN,NaN,NaN,...,2.0,NaN,NaN,0.068,13,hitting,NaN,NaN,NaN,2017


In [182]:
#okay, let's calculate something new!!! Batting Average!
df['ba'] = df.h / df.ab

In [184]:
# Now We can generate groups
# We can make a compound group by name and stat type
by_player = df.groupby(('name', 'stat_type'))
# and by year
by_year = df.groupby('yr')
# and by type
by_type = df.groupby('stat_type')

In [185]:
#sort the ba stats by year
by_type.get_group('hitting').groupby('yr').mean().ba

yr
2015    0.215686
2016    0.219881
2017    0.241031
Name: ba, dtype: float64

In [186]:
by_player.mean().h / by_player.mean().ab

name                  stat_type
Ashton McGee          hitting      0.327354
Austin Bergner        hitting           NaN
                      pitching          NaN
Bo Weiss              hitting           NaN
                      pitching          NaN
Brandon Martorano     hitting      0.068182
Brandon Riley         hitting      0.306220
Brendan Illies        hitting      0.210526
Brett Daniels         hitting           NaN
                      pitching          NaN
Cody Roberts          hitting      0.262295
                      pitching          NaN
Gianluca Dalatri      hitting           NaN
                      pitching          NaN
Hansen Butler         hitting           NaN
                      pitching          NaN
Ike Freeman           hitting      0.173913
Jason Morgan          hitting           NaN
                      pitching          NaN
Josh Hiatt            hitting           NaN
                      pitching          NaN
Josh Ladowski         hitting      0.269231


In [188]:
#let's format it here
df.loc[:, ('name', 'yr', 'ba')]

,name,yr,ba
0,Brandon Riley,2016,0.293814
1,Brandon Riley,2017,0.316964
2,Kyle Datres,2016,0.250000
3,Kyle Datres,2017,0.265217
4,Brandon Martorano,2017,0.068182
5,Ashton McGee,2017,0.327354
6,Ike Freeman,2017,0.173913
7,Zack Gahagan,2015,0.215686
8,Zack Gahagan,2016,0.297297
9,Zack Gahagan,2017,0.242857
